In [3]:
import  keras
from  keras.datasets  import cifar10
import numpy as np
#from keras.applications.inception_v3 import InceptionV3, preprocess_input
import scipy
from scipy import misc
import os

Data can be downloaded from https://d17h27t6h515a5.cloudfront.net/topher/2016/November/5834b432_vgg-100/vgg-100.zip
Make a virtual environment having Python 3 and do as (train.p and  val.p are train and val files respectively) :

import pickle
with open("train.p", "rb") as f:
    w = pickle.load(f)
pickle.dump(w, open("train.pkl","wb"), protocol=2)
with open("val.p", "rb") as f:
    v = pickle.load(f)
pickle.dump(v, open("val.pkl","wb"), protocol=2)


### Reading bottleneck features 

In [20]:
import pickle
def load_bottleneck_data(training_file, validation_file):
    f = open(training_file, 'r')   # 'r' for reading; can be omitted
    train_data = pickle.load(f)         # load file content as mydict
    f.close()
    g= open(validation_file, 'r')   # 'r' for reading; can be omitted
    validation_data = pickle.load(g)         # load file content as mydict
    g.close() 
    X_train = train_data['features']
    y_train = train_data['labels']
    X_val = validation_data['features']
    y_val = validation_data['labels']
    return X_val, y_val,X_train, y_train



In [21]:
X_train, y_train, X_val, y_val=load_bottleneck_data('train.pkl','val.pkl')
print "X_train shape is "+str(X_train.shape)
print "Y_train shape is "+str(y_train.shape)


X_train shape is (10000, 1, 1, 512)
Y_train shape is (10000, 1)


In [22]:
num_classes = len(np.unique(y_train))
print "Num classes are "+str(num_classes)

Num classes are 10


In [28]:
from keras.callbacks import ModelCheckpoint   
from keras.layers import Input, Flatten, Dense
from keras.models import Model
input_shape = X_train.shape[1:]
inp = Input(shape=input_shape)
x = Flatten()(inp)
x = Dense(num_classes, activation='softmax')(x)
model = Model(inp, x)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='model.best.hdf5', 
                               verbose=1, save_best_only=True)
model.fit(X_train, y_train, batch_size=50, epochs=50,
          validation_split=0.2, callbacks=[checkpointer],
          verbose=2, shuffle=True)


Train on 8000 samples, validate on 2000 samples
Epoch 1/50
 - 1s - loss: 1.7431 - acc: 0.5038 - val_loss: 0.9653 - val_acc: 0.6865

Epoch 00001: val_loss improved from inf to 0.96531, saving model to model.best.hdf5
Epoch 2/50
 - 1s - loss: 0.7803 - acc: 0.7342 - val_loss: 0.7414 - val_acc: 0.7605

Epoch 00002: val_loss improved from 0.96531 to 0.74144, saving model to model.best.hdf5
Epoch 3/50
 - 0s - loss: 0.5968 - acc: 0.7976 - val_loss: 0.6509 - val_acc: 0.7855

Epoch 00003: val_loss improved from 0.74144 to 0.65086, saving model to model.best.hdf5
Epoch 4/50
 - 0s - loss: 0.5038 - acc: 0.8292 - val_loss: 0.6178 - val_acc: 0.7940

Epoch 00004: val_loss improved from 0.65086 to 0.61782, saving model to model.best.hdf5
Epoch 5/50
 - 0s - loss: 0.4545 - acc: 0.8456 - val_loss: 0.5844 - val_acc: 0.8075

Epoch 00005: val_loss improved from 0.61782 to 0.58442, saving model to model.best.hdf5
Epoch 6/50
 - 0s - loss: 0.4097 - acc: 0.8590 - val_loss: 0.5620 - val_acc: 0.8115

Epoch 00006:

In [29]:
model.load_weights('model.best.hdf5')

# evaluate test accuracy
score = model.evaluate(X_val, y_val, verbose=1)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy: %.4f%%' % accuracy)

1000/1000 [==============================] - 0s 79us/step
Test accuracy: 83.1000%
